In [1]:
from libdl import Model, DenseLayer, ReLU
import torch
import numpy as np

In [2]:
model = Model(2, 1.0, 4, int(45000/4), "", "cross_entropy", 10)

model.add(DenseLayer(784, 400, "dense1", 10))
model.add(ReLU())
model.add(DenseLayer(400, 200, "dense2", 10))
model.add(ReLU())
model.add(DenseLayer(200, 10, "dense3", 10))
model.add(ReLU())


In [3]:
#Download mnist and feed it to the network
from tensorflow.keras.datasets import mnist

(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

train_data = np.reshape(train_data.shape[0], -1)

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
from libdl import TensorWrapper
data = TensorWrapper(60000, 28, 28, 1, False)
labels = TensorWrapper(60000, 1, 1, 1, False)

train_data = train_data.astype('float64')
train_labels = train_labels.astype('float64')

In [ ]:
epochs = 5
lr = 0.0001
batch_size = 16
batch = 0

data.set_tensor(train_data[batch:batch + batch_size], 28, 28, 1)

predictions = model.forward(data)

        